# ALGORITHMIC FLOW TO PARSE CURRENCY

**Import statements**

In [1]:
import img2pdf
import re
import math
import numpy as np
import pandas as pd
import imutils
import cv2
import os
import io
from google.cloud import vision
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings("ignore")

## 1. Load images

In [3]:
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname( "__file__" ), '..'))
DATA_DIR = os.path.join(BASE_DIR, "data", "aligned_samples")

In [4]:
list_of_images_paths = [os.path.join(DATA_DIR, file) for file in os.listdir(DATA_DIR) 
                       if file.endswith(".jpg") or file.endswith(".png")]

In [5]:
invoice_images = [cv2.imread(img_path) for img_path in list_of_images_paths]

## 2. Extract OCR results of Google Vision API

In [6]:
def merge_texts(symbols):
    text = ""
    for symbol in symbols:
        text += symbol.text
    return text 

def get_document_bounds(image_file):
    """
    Returns document bounds given an image.
    """
    client = vision.ImageAnnotatorClient()

    bounds = []
    texts = []

    with io.open(image_file, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image, 
                                              image_context={"language_hints": ["bn"]})
    document = response.full_text_annotation

    # Collect word level bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    bounds.append(word.bounding_box)
                    texts.append(merge_texts(word.symbols))


    # The list `bounds` contains the coordinates of the bounding boxes.
    # The list `texts` contains the corresponding texts
    return bounds, texts

In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/ssl/Downloads/mapping-customers-47a86d6bd7ea.json'

In [8]:
doc_bounds = []
doc_texts = []
for n,image_file in enumerate(list_of_images_paths):
    bounds, texts = get_document_bounds(image_file)
    doc_bounds.append(bounds)
    doc_texts.append(texts)

## 3. Store the spatial location of each document text in pandas dataframe

### 3.1 Functions to compute positions of text

In [9]:
def compute_top_y(point):
    return int((point.top_left_y + point.top_right_y)//2)

def compute_bottom_y(point):
    return int((point.bottom_left_y + point.bottom_right_y)//2)

def compute_left_x(point):
    return int((point.top_left_x + point.bottom_left_x)//2)

def compute_right_x(point):
    return int((point.top_right_x + point.bottom_right_x)//2)

def compute_mid_x(point):
    mid_x_1 = (point.top_left_x + point.top_right_x)/2
    mid_x_2 = (point.bottom_left_x + point.bottom_right_x)/2
    mid_x = int((mid_x_1+mid_x_2)//2)
    return mid_x

def compute_mid_y(point):
    mid_y_1 = (point.top_left_y + point.top_right_y)/2
    mid_y_2 = (point.bottom_left_y + point.bottom_right_y)/2
    mid_y = int((mid_y_1+mid_y_2)//2)
    return mid_y

def compute_height(point):
    height_1 = point.bottom_left_y - point.top_left_y
    height_2 = point.bottom_right_y - point.top_right_y
    height = int((height_1+height_2)//2)
    return height

def compute_width(point):
    widht_1 = point.top_right_x - point.top_left_x
    widht_2 = point.bottom_right_x - point.bottom_left_x
    widht = int((widht_1+widht_2)//2)
    return widht

### 3.2 Create the pandas dataframe

In [10]:
doc_dfs = []
for texts, bounds in zip(doc_texts,doc_bounds):
    top_left_x = []
    top_left_y = []
    top_right_x = []
    top_right_y = []
    bottom_right_x = []
    bottom_right_y = []
    bottom_left_x = []
    bottom_left_y = []
    for bound in bounds:
        top_left_x.append(bound.vertices[0].x)
        top_left_y.append(bound.vertices[0].y)
        top_right_x.append(bound.vertices[1].x)
        top_right_y.append(bound.vertices[1].y)
        bottom_right_x.append(bound.vertices[2].x)
        bottom_right_y.append(bound.vertices[2].y)
        bottom_left_x.append(bound.vertices[3].x)
        bottom_left_y.append(bound.vertices[3].y)
    df = pd.DataFrame({"word": texts,
                       "top_left_x":top_left_x,
                       "top_left_y":top_left_y,
                       "top_right_x":top_right_x,
                       "top_right_y":top_right_y,
                       "bottom_right_x":bottom_right_x,
                       "bottom_right_y":bottom_right_y,
                       "bottom_left_x":bottom_left_x,
                       "bottom_left_y":bottom_left_y})
    
    df["top_y"] = df.apply(compute_top_y, axis = 1)
    df["bottom_y"] = df.apply(compute_bottom_y, axis = 1)
    df["left_x"] = df.apply(compute_left_x, axis = 1)
    df["right_x"] = df.apply(compute_right_x, axis = 1)
    df["mid_x"] = df.apply(compute_mid_x, axis = 1)
    df["mid_y"] = df.apply(compute_mid_y, axis = 1)
    df["height"] = df.apply(compute_height, axis = 1)
    df["width"] = df.apply(compute_width, axis = 1)
    doc_dfs.append(df)

## 4. Parse total amount of document from pandas df

### 4.1 Functions containing algorithmic flow to parse total from pandas df

In [11]:
def search_total_word(point, search_word_list = ["total", "amount"]):
    if point.word.lower() in search_word_list:
        return True
    else:
        return False
    
def filter_nearby_words(point, y_upper_limit, y_lower_limit, x_lower_limit):
    if point.mid_y>= y_upper_limit and point.mid_y<=y_lower_limit and point.right_x>x_lower_limit:
        return True
    else:
        return False
    
def compute_distance(point,x1,y1):
    x2 = point.mid_x
    y2 = point.mid_y
    return math.sqrt(((x2-x1)**2)+((y2-y1)**2))

def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

def search_total_by_word(doc_df, search_word = "total"):
    """
    Search total amount in document if it exists
    """
    total_df = doc_df[doc_df.apply(search_total_word,
                                   axis = 1)]
    bottom_y_max = doc_df.bottom_y.max()
    if len(total_df) >= 1:
        right_x = int(total_df.iloc[-1].right_x)
        mid_x = int(total_df.iloc[-1].mid_x)
        mid_y = int(total_df.iloc[-1].mid_y)
        top_y = int(total_df.iloc[-1].top_y)
        bottom_y = int(total_df.iloc[-1].bottom_y)
        height = int(total_df.iloc[-1].height)
        width = int(total_df.iloc[-1].width)
        x_lower_limit = int(right_x)
        y_upper_limit = int(mid_y-0.95*height)
        y_lower_limit = int(mid_y+0.95*height)
        
        if bottom_y>=0.5*bottom_y_max:
            search_df = doc_df[doc_df.apply(filter_nearby_words,
                                                    args=(y_upper_limit,y_lower_limit,x_lower_limit),
                                                    axis = 1)]
            if len(search_df) == 0:
                return False
            search_df["dist"] = search_df.apply(compute_distance,
                                                args=(mid_x,mid_y),
                                                axis = 1)
            search_df = search_df.sort_values("dist")
            total = False
            for row in search_df.iterrows():
                word = row[1].word
                if hasNumbers(str(word)):
                    total = str(word)
            return total
    else:
        return False
    
    return False
    
def search_total(doc_df):
    result = search_total_by_word(doc_df)
    if result:
        return result
    else:
        return False

### 4.2 Display the results of invoice corpus

In [12]:
for doc_df, path in zip(doc_dfs, list_of_images_paths):
    result = search_total(doc_df)
    if result:
        print("Filename: {} | Total:{}".format(path.split("/")[-1],result))
    else:
        print("Filename: {} | Total not found".format(path.split("/")[-1]))

Filename: local_14.jpg | Total:35,518.35
Filename: local_11.jpg | Total not found
Filename: local_9.jpg | Total:50,161.00
Filename: local_1.jpg | Total:1,450.00
Filename: local_2.jpg | Total:357,075.53
Filename: local_4.jpg | Total:65,000.00
Filename: local_6.jpg | Total:56,700.00
Filename: local_10.jpg | Total:136,535.00
Filename: loval_7.jpg | Total:1,558,904.00
Filename: net_1.png | Total:154.06
Filename: local_15.jpg | Total:168,819.00
Filename: local_13.jpg | Total:6,300.00
Filename: local_8.jpg | Total:8,004.00
Filename: local_12.jpg | Total:1,555,902.00
Filename: local_5.jpg | Total:79,610.00
Filename: local_3.jpg | Total:47,500.00
